In [121]:
from Bio import Phylo
import matplotlib.pylab as plt
%matplotlib inline
import re
import pandas as pd
import subprocess
from Bio import SeqIO
import sys

In [122]:
sys.path.append('/home/richard/research/tools/')

In [123]:
import moduleIntron

In [2]:
text = ''
with open("/home/richard/research/tests/Nov16_JGI_Tree_building/full_mare_full_core_genes_mapped_v3.phy") as f:
    text = f.read()
    
new_text = re.sub('\s', '_', text)
with open('/home/richard/research/tests/Nov16_JGI_Tree_building/tree_add_underscore.phy','w') as f: f.write(new_text[:-1])

In [3]:
tree = Phylo.read("/home/richard/research/tests/Nov16_JGI_Tree_building/tree_add_underscore.phy", "newick")

In [4]:
# Phylo.draw_ascii(tree)
# Phylo.draw(tree, branch_labels=lambda c: c.branch_length)
# Phylo.draw_graphviz(tree)

In [23]:
species = sorted(map(lambda x: x.name, tree.get_terminals()))

In [31]:
with open('/home/richard/research/tests/Nov16_JGI_Tree_building/longNames.csv','w') as f:
    for name in species:
        f.write(' '.join(name.split('_'))+',\n')

In [6]:
## what to do ...
# 1. get short names
# 2. get gff files
# 3. get intron
# 4. parse acceptor, donor
# 5. calculate KL distance

In [26]:
info = pd.read_csv('/home/richard/research/data_small/fullTableInfoGff3GffRNAESwithDensity20151109.csv')
info.index = info.shortName

In [27]:
shortNames = []
for name in species:
    name = ' '.join(name.split('_'))
    for shortName in info.index:
        if name in str(info.ix[shortName].longName):
            shortNames.append(shortName)
            break

In [37]:
info.ix[shortNames][['longName']].to_csv('/home/richard/research/tests/Nov16_JGI_Tree_building/shortName_longName.csv')

In [38]:
shortNames

['Ascru1',
 'Aspnid1',
 'Babin1',
 'Batde5',
 'Canar1',
 'Canta1',
 'Cante1',
 'Cryne_H99_1',
 'Cybja1',
 'Debha1',
 'Dekbr2',
 'Ashgo1_1',
 'Hanva1_1',
 'Hypbu1',
 'Klula1',
 'Lipst1_1',
 'Metbi1',
 'Neucr2',
 'Pacta1_2',
 'Picme2',
 'Pneji1',
 'Rhior3',
 'Sacce1',
 'Saico1',
 'Schco3',
 'Schpo1',
 'Spapa3',
 'Tapde1_1',
 'Trire2',
 'Tubme1',
 'Wican1',
 'Yarli1']

In [57]:
## cp gff files
problems = []
target = '/home/richard/research/tests/Nov16_JGI_Tree_building/gff_jgi/'
for shortName in shortNames:
    command = 'cp /home/richard/largeDataSet/fungi_download/{}/{} {}'.format(shortName,info.ix[shortName].gffName, target)
    try:
        subprocess.check_call(command, shell = True)
    except:
        print 'problem with', shortName
        problems.append(shortName)

problem with Batde5
problem with Cante1
problem with Cryne_H99_1
problem with Rhior3
problem with Spapa3


In [47]:
shortNames_filtered = [shortName for shortName in shortNames if shortName not in problems]

In [53]:
jgi = {shortName:{} for shortName in shortNames_filtered}
for shortName in shortNames_filtered:
    jgi[shortName]['gff'] = info.ix[shortName].gffName[:-3]

In [64]:
#convert gff to gtf
target_gtf = '/home/richard/research/tests/Nov16_JGI_Tree_building/gtf_converted/'
for shortName in shortNames_filtered:
    jgi[shortName]['gtf'] = jgi[shortName]['gff'][:-3]+'gtf'
    command = 'python /home/richard/research/tools/JGIgff_to_gtf.py {}{} {}{}'.format(target,jgi[shortName]['gff'], target_gtf, jgi[shortName]['gff'][:-3]+'gtf')
    subprocess.check_call(command, shell = True)

In [77]:
# get intron from gtf files
problems = []
target_intron = '/home/richard/research/tests/Nov16_JGI_Tree_building/introns/'
for shortName in shortNames_filtered:
    jgi[shortName]['gff3'] = jgi[shortName]['gff'][:-3]+'gff3'
    try:
        command1 = '/home/tool/gt/bin/gt gtf_to_gff3 -force -tidy -o {}{} {}{}'.format(target_intron, jgi[shortName]['gff'][:-3]+'gff3', target_gtf, jgi[shortName]['gtf'])
#     print command1
        subprocess.check_call(command1, shell = True)
    
        command2 = '/home/tool/gt/bin/gt gff3 -force -addintrons -o {}{} {}{}'.format(target_intron, 'tmp_'+jgi[shortName]['gff3'],target_intron, jgi[shortName]['gff3'])
        subprocess.check_call(command2, shell = True)
        
        command3 = "grep -P '\\tintron\\t' {}{} > {}{}_introns.gff".format(target_intron, 'tmp_'+jgi[shortName]['gff3'], target_intron, shortName)
#         print command3
        subprocess.check_call(command3, shell = True)
    except:
        print 'problem with', shortName
        problems.append(shortName)

problem with Babin1
problem with Trire2
problem with Wican1


In [79]:
shortNames_filtered = [shortName for shortName in shortNames_filtered if shortName not in problems]

In [83]:
## generate donor, acceptor matrix
target_folder = '/home/richard/research/tests/Nov16_JGI_Tree_building/donor_acceptor/'
for shortName in shortNames_filtered: 
    command = 'mkdir {}'.format(target_folder+shortName)
    subprocess.check_call(command, shell=True)

In [87]:
## cp fasta files
target_fasta = '/home/richard/research/tests/Nov16_JGI_Tree_building/fasta/'
for shortName in shortNames_filtered:
    jgi[shortName]['fasta'] = info.ix[shortName].fastaName[:-3]
    command = 'cp /home/richard/largeDataSet/fungi_download/{}/{} {}'.format(shortName, info.ix[shortName].fastaName, target_fasta)
#     print command
#     subprocess.check_call(command, shell=True)

In [95]:
script = ''
for shortName in shortNames_filtered:
    script += 'cd {}\n'.format(shortName)
    script += 'echo "working on {}"\n'.format(shortName)
    script += '/storage3/w/b/get_don_acc.pl  --seq ../../fasta/{}  --introns  ../../introns/{}_introns.gff\n'.format(jgi[shortName]['fasta'], shortName)
    script += 'cd ..\n'
with open('/home/richard/research/tests/Nov16_JGI_Tree_building/donor_acceptor/get_don_acc.sh','w') as f: f.write(script)

In [93]:
for key in shortNames_filtered:
    print jgi[key]['fasta']

Ascru1_AssemblyScaffolds.fasta
Aspnid1_AssemblyScaffolds.fasta
Canar1_AssemblyScaffolds.fasta
Canta1_AssemblyScaffolds.fasta
Cybja1_AssemblyScaffolds.fasta
Debha1_AssemblyScaffolds.fasta
Dekbr2_AssemblyScaffolds.fasta
Ashgo1_1_AssemblyScaffolds.fasta
Hanva1_1_AssemblyScaffolds.fasta
Hypbu1_AssemblyScaffolds.fasta
Klula1_AssemblyScaffolds.fasta
Lipst1_1_AssemblyScaffolds.fasta
Metbi1_AssemblyScaffolds.fasta
Neucr2_AssemblyScaffolds.fasta
Pacta1_2_AssemblyScaffolds.fasta
Picme2_AssemblyScaffolds.fasta
Pneji1_AssemblyScaffolds.fasta
Sacce1_AssemblyScaffolds.fasta
Saico1_AssemblyScaffolds.fasta
Schco3_AssemblyScaffolds.fasta
Schpo1_AssemblyScaffolds.fasta
Tapde1_1_AssemblyScaffolds.fasta
Tubme1_AssemblyScaffolds.fasta
Yarli1_AssemblyScaffolds.fasta


In [96]:
# Klula1, Yarli1 need to be fixed
source_bad_fasta = '/home/richard/research/tests/Nov16_JGI_Tree_building/problem_fasta/'

In [102]:
Klula1 = list(SeqIO.parse('/home/richard/research/tests/Nov16_JGI_Tree_building/problem_fasta/Klula1_AssemblyScaffolds.fasta', 'fasta'))
Yarli1 = list(SeqIO.parse('/home/richard/research/tests/Nov16_JGI_Tree_building/problem_fasta/Yarli1_AssemblyScaffolds.fasta', 'fasta'))

In [108]:
## change to upper
for i in range(len(Klula1)):
    Klula1[i].seq = Klula1[i].seq.upper()

In [112]:
## change to upper
for i in range(len(Yarli1)):
    Yarli1[i].seq = Yarli1[i].seq.upper()

In [114]:
## write to fasta folder
SeqIO.write(Klula1, '/home/richard/research/tests/Nov16_JGI_Tree_building/fasta/Klula1_AssemblyScaffolds.fasta', 'fasta')

6

In [115]:
SeqIO.write(Yarli1, '/home/richard/research/tests/Nov16_JGI_Tree_building/fasta/Yarli1_AssemblyScaffolds.fasta', 'fasta')

6

In [120]:
print shortNames_filtered

['Ascru1', 'Aspnid1', 'Canar1', 'Canta1', 'Cybja1', 'Debha1', 'Dekbr2', 'Ashgo1_1', 'Hanva1_1', 'Hypbu1', 'Klula1', 'Lipst1_1', 'Metbi1', 'Neucr2', 'Pacta1_2', 'Picme2', 'Pneji1', 'Sacce1', 'Saico1', 'Schco3', 'Schpo1', 'Tapde1_1', 'Tubme1', 'Yarli1']


In [125]:
for shortName in shortNames_filtered:
    jgi[shortName]['intronDensity'] = moduleIntron.get_intron_density(target+jgi[shortName]['gff'])

In [128]:
table = info.ix[shortNames_filtered][['longName','gc']]

In [129]:
intronDensity = [jgi[shortName]['intronDensity'] for shortName in table.index]

In [131]:
table['intronDensity'] = intronDensity

In [133]:
table.to_csv('/home/richard/research/tests/Nov16_JGI_Tree_building/speciesInfo.csv')